# Семинар 12: K-Means, Иерархическая кластеризация и DBSCAN на практике

**Цель семинара:** На простых синтетических данных шаг за шагом разобрать, как работают, где ошибаются и как настраиваются три ключевых алгоритма кластеризации. Мы также научимся интерпретировать полученные кластеры.

In [ ]:
import os
# Решение проблемы с утечкой памяти KMeans на Windows
os.environ['OMP_NUM_THREADS'] = '1'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.neighbors import NearestNeighbors

sns.set_theme(style="whitegrid")

## Часть 1: K-Means — кластеризация на основе центроидов

**Датасет:** `make_blobs` — сгенерированные данные, которые идеально подходят для K-Means (три отдельных "облака" точек).

In [ ]:
X_blobs, y_blobs = make_blobs(n_samples=500, centers=3, random_state=42, cluster_std=1.0)

plt.figure(figsize=(8,6))
sns.scatterplot(x=X_blobs[:,0], y=X_blobs[:,1])
plt.title("Исходные данные (Blobs)")
plt.show()

**Комментарий:** *Перед применением K-Means (и любого другого алгоритма, основанного на расстоянии) **крайне важно масштабировать данные**. Иначе признаки с большим диапазоном значений будут непропорционально сильно влиять на результат.*

In [ ]:
scaler = StandardScaler()
X_blobs_scaled = scaler.fit_transform(X_blobs)

### 1.1. Поиск оптимального K с помощью метода локтя

**Комментарий:** *Мы не знаем, сколько кластеров в данных. Прогоним K-Means в цикле с разным `k` и будем записывать `inertia_` (WCSS). Затем построим график и найдем "локоть".*

In [ ]:
ssd = []
k_range = range(1, 11)

for k in k_range:
    model = KMeans(n_clusters=k, random_state=42, n_init='auto')
    model.fit(X_blobs_scaled)
    ssd.append(model.inertia_)

plt.figure(figsize=(8,6))
plt.plot(k_range, ssd, 'o-')
plt.xlabel("Количество кластеров (K)")
plt.ylabel("Sum of Squared Distances (WCSS)")
plt.title("Метод локтя для K-Means")
plt.xticks(k_range)
plt.show()

**Вывод:** *На графике отчетливо виден "локоть" в точке K=3. Это говорит о том, что оптимальное количество кластеров — три.*

### 1.2. Финальная модель K-Means и интерпретация

**Комментарий:** *Обучим модель с `k=3` и визуализируем результат. Затем посмотрим, как можно интерпретировать полученные кластеры.*

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
labels_kmeans = kmeans.fit_predict(X_blobs_scaled)

plt.figure(figsize=(8,6))
sns.scatterplot(x=X_blobs[:,0], y=X_blobs[:,1], hue=labels_kmeans, palette='viridis')
plt.title("Результат K-Means (K=3)")
plt.show()

#### Интерпретация кластеров
**Комментарий:** *Алгоритм вернул нам метки 0, 1, 2. Сами по себе эти цифры ничего не значат. Наша задача как аналитиков — придать им смысл. Основной способ — сгруппировать исходные данные по меткам кластеров и посмотреть на средние значения признаков.*

In [ ]:
df_blobs = pd.DataFrame(X_blobs, columns=['Feature_1', 'Feature_2'])
df_blobs['Cluster'] = labels_kmeans

# Группируем и считаем средние
cluster_profile = df_blobs.groupby('Cluster').mean()
print(cluster_profile)
plt.figure(figsize=(10, 4))
sns.heatmap(cluster_profile, annot=True, cmap='viridis')
plt.title('Профиль кластеров K-Means')
plt.show()

**Вывод:** *Мы видим, что:
- **Кластер 0** характеризуется высокими значениями обоих признаков.
- **Кластер 1** имеет низкое значение Feature_1 и высокое Feature_2.
- **Кластер 2** имеет низкое значение Feature_2. 
Так мы создаем "профиль" каждого кластера, что позволяет дать им осмысленные названия.*

## Часть 2: Иерархическая кластеризация

**Комментарий:** *Используем небольшой, но понятный датасет, чтобы наглядно построить дендрограмму.*

In [ ]:
# Создаем наглядный датасет (как в лекции)
X_hier = np.array([
    [8, 150], [9, 160], [8.5, 140], [7.5, 155], # Офисные работники
    [3, 40], [4, 50], [2.5, 35], [3.5, 45],    # Студенты
    [5, 200], [6, 210], [4.5, 190], [5.5, 220] # Фрилансеры
])

# Строим дендрограмму
linked = linkage(X_hier, method='ward')

plt.figure(figsize=(12, 8))
dendrogram(linked, orientation='top', labels=[f"Точка {i}" for i in range(len(X_hier))], distance_sort='descending')
plt.title('Дендрограмма иерархической кластеризации')
plt.ylabel('Расстояние по Уорду')
plt.axhline(y=100, c='k', linestyle='--') 
plt.show()

**Вывод:** *Дендрограмма наглядно показывает иерархию объединения точек. Горизонтальный срез на уровне `y=100` пересекает 3 вертикальные линии, что подтверждает нашу гипотезу о наличии 3 кластеров.*

## Часть 3: DBSCAN — кластеризация на основе плотности

**Комментарий:** *Теперь перейдем к данным, где K-Means бессилен.*

In [ ]:
X_moons, y_moons = make_moons(n_samples=500, noise=0.1, random_state=42)
X_moons_scaled = StandardScaler().fit_transform(X_moons)

### 3.1. Сравнение K-Means и DBSCAN

**Комментарий:** *Применим оба алгоритма к данным `moons` и посмотрим на результат.*

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42, n_init='auto')
dbscan = DBSCAN(eps=0.3)

labels_kmeans = kmeans.fit_predict(X_moons_scaled)
labels_dbscan = dbscan.fit_predict(X_moons_scaled)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.scatterplot(ax=axes[0], x=X_moons[:,0], y=X_moons[:,1], hue=labels_kmeans, palette='viridis')
axes[0].set_title('Неудачный результат K-Means')

sns.scatterplot(ax=axes[1], x=X_moons[:,0], y=X_moons[:,1], hue=labels_dbscan, palette='viridis')
axes[1].set_title('Успешный результат DBSCAN')
plt.show()

### 3.2. Подбор гиперпараметров для DBSCAN

**Комментарий:** *Как мы выбрали `eps=0.3`? Это не магия. Существует полезная эвристика для подбора `eps`. Идея в том, чтобы найти "точку перегиба" на графике расстояний до k-го ближайшего соседа.*

1.  Выбираем `min_samples`. Хорошее эмпирическое правило: `min_samples = 2 * количество_признаков`. У нас 2 признака, так что возьмем `min_samples=4`.
2.  Для **каждой** точки в датасете находим расстояние до ее 4-го ближайшего соседа.
3.  Сортируем эти расстояния по возрастанию и строим график.
4.  Ищем на графике "локоть" — точку, где кривая резко устремляется вверх. Оптимальное значение `eps` будет лежать примерно на этом уровне.

In [ ]:
min_samples = 2 * X_moons_scaled.shape[1]

neighbors = NearestNeighbors(n_neighbors=min_samples)
neighbors_fit = neighbors.fit(X_moons_scaled)
distances, indices = neighbors_fit.kneighbors(X_moons_scaled)

# Сортируем расстояния до k-го соседа (k = min_samples)
k_distances = np.sort(distances[:, min_samples-1], axis=0)

plt.figure(figsize=(8,6))
plt.plot(k_distances)
plt.title('График k-расстояний для подбора Epsilon')
plt.xlabel("Точки, отсортированные по расстоянию")
plt.ylabel(f"Расстояние до {min_samples}-го соседа")
plt.axhline(y=0.3, c='r', linestyle='--') # Наш предполагаемый локоть
plt.grid(True)
plt.show()

**Вывод семинара:** *Мы увидели, что нет "лучшего" алгоритма. K-Means быстр и хорош для сферических кластеров. Иерархическая кластеризация дает наглядную дендрограмму. DBSCAN, основанный на плотности, отлично справляется с кластерами сложной формы и выбросами, но требует более тонкой настройки гиперпараметров.*